In [2]:
# forecast_national

import pandas as pd
import numpy as np
from prophet import Prophet

df = pd.read_csv('../data/processed/monthly_sales_national.csv')

# Create Prophet model with floor at 0
model = Prophet(growth='linear', 
               yearly_seasonality=True,
               weekly_seasonality=False,
               daily_seasonality=False,
               seasonality_mode='multiplicative')  # multiplicative seasonality helps prevent negative values

model.fit(df)

# Create future dataframe (next 6 months)
future = model.make_future_dataframe(periods=6, freq='ME')

# Forecast
forecast = model.predict(future)

# Ensure no negative values in forecast
forecast['yhat'] = np.maximum(forecast['yhat'], 0)
forecast['yhat_lower'] = np.maximum(forecast['yhat_lower'], 0)
forecast['yhat_upper'] = np.maximum(forecast['yhat_upper'], 0)

# Calculate and print forecast statistics
print("\nForecast Statistics:")
print(f"Minimum predicted value: {forecast['yhat'].min():.2f}")
print(f"Maximum predicted value: {forecast['yhat'].max():.2f}")
print(f"Mean predicted value: {forecast['yhat'].mean():.2f}")

forecast.to_csv('../data/forecast/national_forecast.csv', index=False)

print("\n✅ National forecast saved.")

17:00:07 - cmdstanpy - INFO - Chain [1] start processing
17:00:07 - cmdstanpy - INFO - Chain [1] done processing



Forecast Statistics:
Minimum predicted value: 11006.60
Maximum predicted value: 91526.28
Mean predicted value: 43172.55

✅ National forecast saved.


In [5]:
# forecast_regions

import numpy as np
from prophet import Prophet
df = pd.read_csv('../data/processed/monthly_sales_by_region.csv')

# Forecast for each region
for region in df['Region'].unique():
    print(f"\nProcessing forecast for {region}...")
    
    region_df = df[df['Region'] == region][['ds', 'y']]
    
    # Create Prophet model with optimized parameters
    model = Prophet(growth='linear',
                   yearly_seasonality=True,
                   weekly_seasonality=False,
                   daily_seasonality=False,
                   seasonality_mode='multiplicative')  # multiplicative seasonality helps prevent negative values
    
    model.fit(region_df)

    # Forecast next 6 months
    future = model.make_future_dataframe(periods=6, freq='ME')
    forecast = model.predict(future)
    
    # Ensure no negative values
    forecast['yhat'] = np.maximum(forecast['yhat'], 0)
    forecast['yhat_lower'] = np.maximum(forecast['yhat_lower'], 0)
    forecast['yhat_upper'] = np.maximum(forecast['yhat_upper'], 0)
    
    # Print forecast statistics
    print(f"Forecast Statistics for {region}:")
    print(f"Minimum predicted value: {forecast['yhat'].min():.2f}")
    print(f"Maximum predicted value: {forecast['yhat'].max():.2f}")
    print(f"Mean predicted value: {forecast['yhat'].mean():.2f}")

    # Save forecast for the region
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].to_csv(
        f'../data/forecast/regions/{region}_forecast.csv', 
        index=False
    )

    print(f"✅ Forecast saved for {region}")

print("\nAll regional forecasts completed!")

17:02:50 - cmdstanpy - INFO - Chain [1] start processing
17:02:50 - cmdstanpy - INFO - Chain [1] done processing



Processing forecast for Central...


17:02:50 - cmdstanpy - INFO - Chain [1] start processing


Forecast Statistics for Central:
Minimum predicted value: 0.00
Maximum predicted value: 21162.20
Mean predicted value: 10487.98
✅ Forecast saved for Central

Processing forecast for East...


17:02:50 - cmdstanpy - INFO - Chain [1] done processing
17:02:50 - cmdstanpy - INFO - Chain [1] start processing


Forecast Statistics for East:
Minimum predicted value: 1936.48
Maximum predicted value: 39255.25
Mean predicted value: 13868.92
✅ Forecast saved for East

Processing forecast for South...


17:02:50 - cmdstanpy - INFO - Chain [1] done processing
17:02:50 - cmdstanpy - INFO - Chain [1] start processing


Forecast Statistics for South:
Minimum predicted value: 266.17
Maximum predicted value: 19698.68
Mean predicted value: 8275.28
✅ Forecast saved for South

Processing forecast for West...


17:02:50 - cmdstanpy - INFO - Chain [1] done processing


Forecast Statistics for West:
Minimum predicted value: 2907.96
Maximum predicted value: 35326.63
Mean predicted value: 15246.23
✅ Forecast saved for West

All regional forecasts completed!
